In [1]:
import numpy as np
from scipy import linalg as lg
import time

In [18]:
def count_layers(len_dim, factors):
    layers = []
    layer = []
    for n in range(len_dim):
        layer.append(factors[n].T @ factors[n])
    layers.append(layer)

    for n in range(1, len_dim - 1):
        layer = []
        for r in range(len_dim - n):
            layer.append(layers[n - 1][r] * layers[0][r + n])
        layers.append(layer)
    return layers

In [19]:
def count_W(len_dim, layers):
    W = []
    for row in range(len_dim):
        start_layer = row - 1
        end_layer = len_dim - row - 2
        if start_layer >= 0:
            W_col = layers[start_layer][0]
            if end_layer >= 0:
                W_col = W_col * layers[end_layer][-1]
        else:
            W_col = layers[end_layer][-1]

        W_row = [W_col]

        for col in range(row + 1, len_dim):
            start_layer = row - 1
            middle_layer = col - row - 2
            end_layer = len_dim - row - 2

            if start_layer >= 0:
                W_col = layers[start_layer][0]
                if middle_layer >= 0:
                    W_col = W_col * layers[middle_layer][row + 1]
                if end_layer >= 0:
                    W_col = W_col * layers[end_layer][-1]
            elif middle_layer >= 0:
                W_col = layers[middle_layer][row + 1]
                if end_layer >= 0:
                    W_col = W_col * layers[end_layer][-1]
            else:
                W_col = layers[end_layer][-1]


            W_row.append(W_col)

        W.append(W_row)

    return W

In [106]:
def count_JJ_d(len_dim, W, pos, jacob_size, dimensions, rank, factors, coef):
    JJ = np.zeros((jacob_size, jacob_size))
    for row in range(len_dim):
        BB = W[row][0]
        for r in range(dimensions[row]):
            now_pos = pos[row] + r * rank
            JJ[now_pos:now_pos+rank, now_pos:now_pos+rank] = BB

    for alpha in range(len_dim):
        now_alpha = pos[alpha]
        for i in range(alpha + 1, len_dim):
            now_i = pos[i]

            now_W = W[alpha][i - alpha]
            for betta in range(dimensions[alpha]):
                now_a_b = now_alpha + betta * rank
                for j in range(dimensions[i]):
                    now_i_j = now_i + j * rank
                    for gamma in range(rank):
                        now_a_b_g = now_a_b + gamma
                        for k in range(rank):
                            now_i_j_k = now_i_j + k
                            JJ[now_a_b_g, now_i_j_k] = now_W[gamma][k] * factors[i][j][gamma] * factors[alpha][betta][k]
                            JJ[now_i_j_k, now_a_b_g] = now_W[k][gamma] * factors[i][j][gamma] * factors[alpha][betta][k]


    for i in range(jacob_size):
        JJ[i][i] *= (1 + coef)

    return JJ

In [107]:
def count_J_A(len_dim, factors, rank, dimensions):
    J_start = [factors[0]]
    J_end = [factors[-1]]
    for i in range(1, len_dim - 1):
        J_start.append(lg.khatri_rao(J_start[i - 1], factors[i]))
        J_end.append(lg.khatri_rao(factors[-i - 1], J_end[i - 1]))
    A = lg.khatri_rao(J_start[-1], factors[len_dim - 1])
    A = np.sum(A, axis=1)

    J = np.zeros((np.prod(dimensions), np.sum(dimensions) * rank))
    for k in range(len_dim):
        pos = int(np.sum(dimensions[:k]) * rank)
        for i in range(dimensions[k]):
            E = np.zeros((dimensions[k], rank))
            E[i, :] = np.ones((1, rank))
            if k > 0:
                val = lg.khatri_rao(J_start[k - 1], E)
                if k < len_dim - 1:
                    val = lg.khatri_rao(val, J_end[len_dim - k - 2])
                J[:, pos + i * rank: pos + i * rank + rank] = val
            else:
                J[:, (pos + i * rank):(pos + i * rank + rank)] = lg.khatri_rao(E, J_end[len_dim - 2])

    return J, A

In [199]:
def Levenberg(tensor, rank, seed=42, tol = 10**(-8), n_iteration = 100):

    dimensions = tensor.shape
    len_dim = len(dimensions)

    jacob_size = int(np.sum(dimensions) * rank)

    rng = np.random.RandomState(seed)
    factors = np.array([rng.rand(dimensions[i], rank) for i in range(len_dim)], dtype=object)
    for fuck in factors:
        print(fuck)
        print()
    print()

    eps = 1.0
    it = 0

    pos = [0]
    for i in range(len_dim - 1):
        pos.append(pos[i] + dimensions[i] * rank)

    while eps > tol and it < n_iteration:

        layers = count_layers(len_dim, factors)
        W = count_W(len_dim, layers)
        JJ_d = count_JJ_d(len_dim, W, pos, jacob_size, dimensions, rank, factors, coef=1)
        J, A = count_J_A(len_dim, factors, rank, dimensions)
        print(J)

        # JJ_d = J.T @ J

        # print(np.linalg.matrix_norm(J.T @ J - JJ_d)  / np.linalg.matrix_norm(J))

        # for i in range(jacob_size):
        #     JJ_d[i][i] *= 2

        delta = np.linalg.pinv(JJ_d) @ (J.T @ (A - tensor.reshape((1, -1))).T)

        for i in range(len_dim):
            position = int(np.sum(dimensions[:i]) * rank)
            for r in range(rank):
                factors[i][:, r] -= (delta[position + r * dimensions[i]:position + (r + 1) * dimensions[i]]).reshape(-1,)

        eps = np.linalg.matrix_norm(delta)
        print(eps)


        it += 1

    return factors, it

In [205]:
sizes = np.array((3, 3))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        T[i, j] = np.sin(i + j)

In [206]:
rank = 2
factors, it = Levenberg(T, rank)

[[0.3745401188473625 0.9507143064099162]
 [0.7319939418114051 0.5986584841970366]
 [0.15601864044243652 0.15599452033620265]]

[[0.05808361216819946 0.8661761457749352]
 [0.6011150117432088 0.7080725777960455]
 [0.020584494295802447 0.9699098521619943]]


[[0.05808361 0.86617615 0.         0.         0.         0.
  0.37454012 0.95071431 0.         0.         0.         0.        ]
 [0.60111501 0.70807258 0.         0.         0.         0.
  0.         0.         0.37454012 0.95071431 0.         0.        ]
 [0.02058449 0.96990985 0.         0.         0.         0.
  0.         0.         0.         0.         0.37454012 0.95071431]
 [0.         0.         0.05808361 0.86617615 0.         0.
  0.73199394 0.59865848 0.         0.         0.         0.        ]
 [0.         0.         0.60111501 0.70807258 0.         0.
  0.         0.         0.73199394 0.59865848 0.         0.        ]
 [0.         0.         0.02058449 0.96990985 0.         0.
  0.         0.         0.         0.  

In [207]:
len_dim = len(sizes)
sweep_l = np.ones(rank).reshape(1, -1)

for j in range(len_dim):
    sweep_l = lg.khatri_rao(sweep_l, factors[j])

print(np.linalg.norm(np.sum(sweep_l, axis=1) - T.reshape(-1,)) / np.linalg.norm(T.reshape(-1,)))

108.27936626790354
